In [1]:
import subprocess
import os

netlist_template = """

*CONDUCTION NMOS W1

.INCLUDE ../../model/32nm_HP.pm
.OPTIONS GMIN=1e-020 ABSTOL=1e-18

*Definizione dei parametri
.PARAM Lmin=22n
.PARAM Wmin=22n
.PARAM Ldiff=44n

.TEMP 85

.PARAM N=1

*Descrizione della prima cella NMOS
Mp1 drain gate1 source_inter body1 pmos W={{Wmin*N}} L={{Lmin}} AS={{Wmin*N*Ldiff}} AD={{Wmin*N*Ldiff}} PS={{2*(Ldiff+Wmin*N)}} PD={{2*(Ldiff+Wmin*N)}}

*Descrizione della seconda cella NMOS in series
Mp2 drain_inter gate2 source body2 pmos W={{Wmin*N}} L={{Lmin}} AS={{Wmin*N*Ldiff}} AD={{Wmin*N*Ldiff}} PS={{2*(Ldiff+Wmin*N)}} PD={{2*(Ldiff+Wmin*N)}}

Vd 	drain	0		0
Vg1 	gate1	{inp1}		0
Vg2 	gate2	{inp2}		0
Vs 	source	{source_v}		0
Vb1		body1	alim		0
Vb2 	body2	alim		0
Vdd 	alim	0		1.0
vO source_inter drain_inter 0


.CONTROL
dc TEMP 85 86 10
print V(drain_inter) 

.ENDC
.END


"""

inp1s = [0, 'alim', 'alim', 0]
inp2s = [0, 'alim', 0, 'alim']

output_dir = "diff_inputs"

# make the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

outs = []
for inp1, inp2 in zip(inp1s, inp2s):
    netlist_content = netlist_template.format(source_v='alim', inp1=inp1, inp2=inp2)
    
    filename = f"{output_dir}/pmos_{inp1},{inp2}.net"
    
    with open(filename, 'w') as file:
        file.write(netlist_content)
    
    p = subprocess.run(["ngspice", filename] , encoding='utf-8', 
stdout=subprocess.PIPE)
    windstr = p.stdout
    outs.append(windstr)

In [2]:

# make a matrix out of every 10-18 lines
matrix = []
for o in outs:
    lines = o.split('\n')
    matrix.append(lines[10])
    
print(matrix)

['v(drain_inter) = 5.126065e-01', 'v(drain_inter) = 5.339468e-01', 'v(drain_inter) = 5.649137e-01', 'v(drain_inter) = 4.773071e-01']


In [3]:
inp1s = [0, 1, 1, 0]
inp2s = [0, 1, 0, 1]

filename = f"outputs.csv"
with open(filename, 'w') as file:
    file.write("inp1,inp2,V(Internal node)\n")
    for i, line in enumerate(matrix):
        file.write(f"{inp1s[i]},{inp2s[i]},{line.split(' ')[-1]}\n")